# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [43]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [44]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [45]:
import problem_unittests as tests
from collections import Counter
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = [word for word in text]
    vocab_count = Counter(vocab)
    vocab_to_int = {word[0]:idx for idx, word in enumerate(list(vocab_count.most_common()))}
    int_to_vocab = {idx:word for word, idx in vocab_to_int.items()}
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [46]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_dict = {'.': '||Period||',
                  ',': '||Comma||',
                  '"': '||Quotation_Mark||',
                  ';': '||Semicolon||',
                  '!': '||Exclamation_Mark||',
                  '?': '||Question_Mark||',
                  '(': '||Left_Parentheses||',
                  ')': '||Right_Parentheses||',
                  '-': '||Dash||',
                  '\n': '||Return||'}   
    return token_dict
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [47]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [3]:
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    n_batches = len(words)//(sequence_length * batch_size)
    words = np.array(words[:(n_batches*sequence_length*batch_size)])
    

    features, targets = [], []
    for i in range(0, len(words), sequence_length):
        feature = words[i:i+sequence_length]
        target = np.zeros_like(feature)
        try:
            target = words[i+sequence_length]
        except:
            target = words[0]
        
        features.append(feature)
        targets.append(target)
        
    #convert to tensors
    feature_tensors, target_tensors = torch.from_numpy(np.array(features)), torch.from_numpy(np.array(targets))
    
    data = TensorDataset(feature_tensors, target_tensors)
    data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size, shuffle = True)
    # return a dataloader
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [4]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 20,  21,  22,  23,  24],
        [  0,   1,   2,   3,   4],
        [ 25,  26,  27,  28,  29],
        [ 30,  31,  32,  33,  34],
        [ 35,  36,  37,  38,  39],
        [ 15,  16,  17,  18,  19],
        [ 10,  11,  12,  13,  14],
        [ 40,  41,  42,  43,  44],
        [  5,   6,   7,   8,   9],
        [ 45,  46,  47,  48,  49]])

torch.Size([10])
tensor([ 25,   5,  30,  35,  40,  20,  15,  45,  10,   0])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [5]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.output_size = output_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        # define model layers
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim, self.n_layers, batch_first=True, dropout = dropout)
        self.fc = nn.Linear(self.hidden_dim, self.output_size)
       
        self.dropout = nn.Dropout(p=dropout)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function  
        batch_size = nn_input.shape[0]
        embed_input = self.embed(nn_input)
        lstm_output, hidden = self.lstm(embed_input, hidden)
        lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)
        output = self.dropout(lstm_output)
        output = self.fc(output)
        
        output = output.view(batch_size, -1, self.output_size)
        out = output[:, -1]
        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        if (train_on_gpu):
            hidden = (next(self.parameters()).data.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  next(self.parameters()).data.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (next(self.parameters()).data.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      next(self.parameters()).data.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [6]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    train_on_gpu = torch.cuda.is_available()
    if (train_on_gpu):
           rnn, inp, target = rnn.cuda(), inp.cuda(), target.cuda()
    # perform backpropagation and optimization
    hidden = tuple([each.data for each in hidden])
    optimizer.zero_grad()
    output, hidden = rnn(inp, hidden)
    loss = criterion(output.squeeze(), target)
    loss.backward()
    optimizer.step()
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [8]:
print(len(int_text))

892110


In [9]:
# Data params
# Sequence Length
sequence_length = 6 # of words in a sequence
# Batch Size
batch_size = 64

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [10]:
# Training parameters
# Number of Epochs
num_epochs = 30
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int) 
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 500
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 10

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 30 epoch(s)...
Epoch:    1/30    Loss: 9.084050607681274

Epoch:    1/30    Loss: 6.995461559295654

Epoch:    1/30    Loss: 6.652137517929077

Epoch:    1/30    Loss: 6.269296312332154

Epoch:    1/30    Loss: 6.156131982803345

Epoch:    1/30    Loss: 5.983782958984375

Epoch:    1/30    Loss: 6.086099529266358

Epoch:    1/30    Loss: 5.864189195632934

Epoch:    1/30    Loss: 6.206787014007569

Epoch:    1/30    Loss: 5.815178251266479

Epoch:    1/30    Loss: 5.874624156951905

Epoch:    1/30    Loss: 5.590088748931885

Epoch:    1/30    Loss: 5.80139012336731

Epoch:    1/30    Loss: 5.8248138427734375

Epoch:    1/30    Loss: 5.81760950088501

Epoch:    1/30    Loss: 5.4485344886779785

Epoch:    1/30    Loss: 5.7600555419921875

Epoch:    1/30    Loss: 5.5437352657318115

Epoch:    1/30    Loss: 5.3279420852661135

Epoch:    1/30    Loss: 5.54026951789856

Epoch:    1/30    Loss: 5.70044322013855

Epoch:    1/30    Loss: 5.886346292495728

Epoch:    1/30    Loss: 5

Epoch:    1/30    Loss: 4.890138721466064

Epoch:    1/30    Loss: 4.910463953018189

Epoch:    1/30    Loss: 4.929575443267822

Epoch:    1/30    Loss: 4.96461820602417

Epoch:    1/30    Loss: 4.78700635433197

Epoch:    1/30    Loss: 4.730292940139771

Epoch:    1/30    Loss: 4.921347379684448

Epoch:    1/30    Loss: 4.662320280075074

Epoch:    1/30    Loss: 5.032066774368286

Epoch:    1/30    Loss: 4.687074756622314

Epoch:    1/30    Loss: 4.856114435195923

Epoch:    1/30    Loss: 4.982973861694336

Epoch:    1/30    Loss: 5.052369165420532

Epoch:    1/30    Loss: 4.748007106781006

Epoch:    1/30    Loss: 4.7107847213745115

Epoch:    1/30    Loss: 4.678832483291626

Epoch:    1/30    Loss: 4.732467079162598

Epoch:    1/30    Loss: 4.891993236541748

Epoch:    1/30    Loss: 4.686230421066284

Epoch:    1/30    Loss: 4.672486734390259

Epoch:    1/30    Loss: 4.765953254699707

Epoch:    1/30    Loss: 4.784014344215393

Epoch:    1/30    Loss: 4.756644773483276

Epoch:    1/

Epoch:    2/30    Loss: 4.691123199462891

Epoch:    2/30    Loss: 4.601697683334351

Epoch:    2/30    Loss: 4.431751251220703

Epoch:    2/30    Loss: 4.697372722625732

Epoch:    2/30    Loss: 4.517445707321167

Epoch:    2/30    Loss: 4.639889240264893

Epoch:    2/30    Loss: 4.808301877975464

Epoch:    2/30    Loss: 4.339040517807007

Epoch:    2/30    Loss: 4.553295421600342

Epoch:    2/30    Loss: 4.63047513961792

Epoch:    2/30    Loss: 4.205301094055176

Epoch:    2/30    Loss: 4.499992847442627

Epoch:    2/30    Loss: 4.658674192428589

Epoch:    2/30    Loss: 4.565420341491699

Epoch:    2/30    Loss: 4.403257036209107

Epoch:    2/30    Loss: 4.481516170501709

Epoch:    2/30    Loss: 4.500877141952515

Epoch:    2/30    Loss: 4.396097898483276

Epoch:    2/30    Loss: 4.715815782546997

Epoch:    2/30    Loss: 4.702153873443604

Epoch:    2/30    Loss: 4.443569231033325

Epoch:    2/30    Loss: 4.670051097869873

Epoch:    2/30    Loss: 4.445018315315247

Epoch:    2/

Epoch:    3/30    Loss: 4.697815489768982

Epoch:    3/30    Loss: 4.219480323791504

Epoch:    3/30    Loss: 4.301202321052552

Epoch:    3/30    Loss: 4.594737720489502

Epoch:    3/30    Loss: 4.379376292228699

Epoch:    3/30    Loss: 4.279119277000428

Epoch:    3/30    Loss: 4.224784970283508

Epoch:    3/30    Loss: 4.409577822685241

Epoch:    3/30    Loss: 4.18730411529541

Epoch:    3/30    Loss: 4.5269163131713865

Epoch:    3/30    Loss: 4.393619132041931

Epoch:    3/30    Loss: 4.233287763595581

Epoch:    3/30    Loss: 4.717074060440064

Epoch:    3/30    Loss: 4.34702570438385

Epoch:    3/30    Loss: 4.575814747810364

Epoch:    3/30    Loss: 4.349825143814087

Epoch:    3/30    Loss: 4.25186333656311

Epoch:    3/30    Loss: 4.362956476211548

Epoch:    3/30    Loss: 4.4029250860214235

Epoch:    3/30    Loss: 4.547167086601258

Epoch:    3/30    Loss: 4.417986583709717

Epoch:    3/30    Loss: 4.2847585439682

Epoch:    3/30    Loss: 4.435477900505066

Epoch:    3/30

Epoch:    4/30    Loss: 4.160111308097839

Epoch:    4/30    Loss: 4.290582299232483

Epoch:    4/30    Loss: 4.255142331123352

Epoch:    4/30    Loss: 4.172678518295288

Epoch:    4/30    Loss: 4.323028564453125

Epoch:    4/30    Loss: 4.428517389297485

Epoch:    4/30    Loss: 4.256661725044251

Epoch:    4/30    Loss: 4.314753484725952

Epoch:    4/30    Loss: 4.505028963088989

Epoch:    4/30    Loss: 4.452885627746582

Epoch:    4/30    Loss: 4.188143610954285

Epoch:    4/30    Loss: 4.239332365989685

Epoch:    4/30    Loss: 4.129481339454651

Epoch:    4/30    Loss: 4.082048678398133

Epoch:    4/30    Loss: 4.215937399864197

Epoch:    4/30    Loss: 4.107482075691223

Epoch:    4/30    Loss: 4.250562381744385

Epoch:    4/30    Loss: 4.299384951591492

Epoch:    4/30    Loss: 4.100442719459534

Epoch:    4/30    Loss: 4.133211636543274

Epoch:    4/30    Loss: 4.255440759658813

Epoch:    4/30    Loss: 4.345036745071411

Epoch:    4/30    Loss: 4.539964103698731

Epoch:    4

...

Epoch:   23/30    Loss: 2.9543654680252076

Epoch:   23/30    Loss: 3.0612967014312744

Epoch:   23/30    Loss: 3.3199177503585817

Epoch:   23/30    Loss: 3.1820005416870116

Epoch:   23/30    Loss: 3.1420325756073

Epoch:   23/30    Loss: 3.2400357007980345

Epoch:   23/30    Loss: 3.1701319217681885

Epoch:   23/30    Loss: 3.0982603073120116

Epoch:   23/30    Loss: 3.3346981763839723

Epoch:   23/30    Loss: 2.9457332491874695

Epoch:   23/30    Loss: 3.2414873838424683

Epoch:   23/30    Loss: 3.1612799882888796

Epoch:   23/30    Loss: 2.94229896068573

Epoch:   23/30    Loss: 3.069216561317444

Epoch:   23/30    Loss: 3.1417745113372804

Epoch:   23/30    Loss: 3.2345378160476685

Epoch:   23/30    Loss: 3.0616297245025637

Epoch:   23/30    Loss: 3.377500891685486

Epoch:   23/30    Loss: 3.0858586549758913

Epoch:   23/30    Loss: 3.179185390472412

Epoch:   23/30    Loss: 3.0009230852127073

Epoch:   23/30    Loss: 3.102643585205078

Epoch:   23/30    Loss: 3.089742231369018

Epoch:   23/30    Loss: 3.091712808609009

Epoch:   23/30    Loss: 3.3067859649658202

Epoch:   23/30    Loss: 3.1629625797271728

Epoch:   23/30    Loss: 3.3508126974105834

Epoch:   23/30    Loss: 3.0380665063858032

Epoch:   23/30    Loss: 3.266673445701599

Epoch:   23/30    Loss: 3.0461442470550537

Epoch:   23/30    Loss: 3.4735946893692016

Epoch:   23/30    Loss: 3.2324275970458984

Epoch:   23/30    Loss: 3.318776822090149

Epoch:   23/30    Loss: 3.1580111503601076

Epoch:   23/30    Loss: 3.2615506649017334

Epoch:   23/30    Loss: 3.204213285446167

Epoch:   23/30    Loss: 3.185917115211487

Epoch:   23/30    Loss: 3.282806468009949

Epoch:   23/30    Loss: 2.9957367181777954

Epoch:   23/30    Loss: 3.121064019203186

Epoch:   23/30    Loss: 3.177745318412781

Epoch:   23/30    Loss: 3.3922178983688354

Epoch:   23/30    Loss: 3.2619096517562864

Epoch:   23/30    Loss: 3.3655827045440674

Epoch:   23/30    Loss: 3.1131542682647706

Epoch:   23/30    Loss: 3.21880414485931

Epoch:   24/30    Loss: 3.135641312599182

Epoch:   24/30    Loss: 3.24179584980011

Epoch:   24/30    Loss: 3.176082491874695

Epoch:   24/30    Loss: 3.2981722831726072

Epoch:   24/30    Loss: 3.360953116416931

Epoch:   24/30    Loss: 3.1519974946975706

Epoch:   24/30    Loss: 3.015295147895813

Epoch:   24/30    Loss: 3.1989445447921754

Epoch:   24/30    Loss: 2.9170986652374267

Epoch:   24/30    Loss: 3.232534623146057

Epoch:   24/30    Loss: 3.3804799795150755

Epoch:   24/30    Loss: 3.2485100269317626

Epoch:   24/30    Loss: 3.1997549533843994

Epoch:   24/30    Loss: 3.328173565864563

Epoch:   24/30    Loss: 3.2600611448287964

Epoch:   24/30    Loss: 3.066431427001953

Epoch:   24/30    Loss: 3.360077166557312

Epoch:   24/30    Loss: 3.1629889249801635

Epoch:   24/30    Loss: 3.285318064689636

Epoch:   24/30    Loss: 3.112757110595703

Epoch:   24/30    Loss: 3.2252721548080445

Epoch:   24/30    Loss: 3.210455393791199

Epoch:   24/30    Loss: 3.301438403129578

Ep

Epoch:   25/30    Loss: 3.2209051132202147

Epoch:   25/30    Loss: 3.134887528419495

Epoch:   25/30    Loss: 3.1853341817855836

Epoch:   25/30    Loss: 3.246865248680115

Epoch:   25/30    Loss: 3.0828939199447634

Epoch:   25/30    Loss: 3.0018104553222655

Epoch:   25/30    Loss: 3.0936338663101197

Epoch:   25/30    Loss: 3.365938496589661

Epoch:   25/30    Loss: 3.1313601970672607

Epoch:   25/30    Loss: 3.0532853841781615

Epoch:   25/30    Loss: 2.898368167877197

Epoch:   25/30    Loss: 3.0684903144836424

Epoch:   25/30    Loss: 3.104542064666748

Epoch:   25/30    Loss: 3.1932831525802614

Epoch:   25/30    Loss: 2.861685800552368

Epoch:   25/30    Loss: 3.0869449853897093

Epoch:   25/30    Loss: 3.138780927658081

Epoch:   25/30    Loss: 3.0230174541473387

Epoch:   25/30    Loss: 3.2426592826843263

Epoch:   25/30    Loss: 3.1913164138793944

Epoch:   25/30    Loss: 3.1021681785583497

Epoch:   25/30    Loss: 3.0600795984268188

Epoch:   25/30    Loss: 3.1615015506744

Epoch:   26/30    Loss: 3.0487446069717405

Epoch:   26/30    Loss: 3.116301345825195

Epoch:   26/30    Loss: 3.1632114171981813

Epoch:   26/30    Loss: 3.1501529693603514

Epoch:   26/30    Loss: 2.9575628519058226

Epoch:   26/30    Loss: 2.9804103136062623

Epoch:   26/30    Loss: 3.1088504076004027

Epoch:   26/30    Loss: 3.0774781465530396

Epoch:   26/30    Loss: 2.988922095298767

Epoch:   26/30    Loss: 3.024422812461853

Epoch:   26/30    Loss: 3.1437524795532226

Epoch:   26/30    Loss: 3.1424004793167115

Epoch:   26/30    Loss: 3.2371719598770143

Epoch:   26/30    Loss: 3.039684510231018

Epoch:   26/30    Loss: 3.1193310260772704

Epoch:   26/30    Loss: 3.0099663972854613

Epoch:   26/30    Loss: 2.823675513267517

Epoch:   26/30    Loss: 2.953162693977356

Epoch:   26/30    Loss: 2.999940538406372

Epoch:   26/30    Loss: 3.2268234968185423

Epoch:   26/30    Loss: 3.112960624694824

Epoch:   26/30    Loss: 2.968586564064026

Epoch:   26/30    Loss: 3.060403919219971

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** I initially tried (sequence_length,batch size) combination of (100, 128) and (100, 64). I assummed a sequence length of 100 was long enough to extract context and short enough to ease computation. These values were with an embedding dimension of int(vocab_size ** 0.25), a formula which I later discovered was for only the skip-gram model. These combinations gave me a minimum loss of 5.42. Then, I decided to use an embedding dimension of 500 (similar to the table comparing embedding dimensions given in the classroom) still with a sequence length and batch size of 100 and 64 respectively and this gave me a minimum loss of 2.71 after 20 epochs. However, I noticed the loss was still decreasing and the format of the script I generated with this model was a bit off, so I increased the number of epochs to 30 which achieved a training loss of 1.78. At the start of setting the hyperparameters, I used 512 hidden dimension and 3 layers lstm (ensuring a lot of feature extraction)just to see how long it takes to train for 10 epochs. Since it didn't take a long period of time and I was already achieving a trainig loss less than 3.5(the maximum loss on the project rubic), I stuck with those values as I increased number of epochs.

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [ ]:
# run the cell multiple times to get different results!
gen_length = 500 # modify the length to your preference
prime_word = "george" # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [24]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.